In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import *

In [5]:
df = pd.read_csv('data_filt_aug.csv') # data concatenate with lang country etc 
df=df.drop_duplicates(subset='user_id')
#Random_value_for_the moment , need to be replaced by the good one.

df['sniper_score']=np.random.rand(len(df))
df['assault_score']=np.random.rand(len(df))
df['support_score']=np.random.rand(len(df))
df['captain_score']=np.random.rand(len(df))
df['strategist_score']=np.random.rand(len(df))

In [6]:
df.head()

,Unnamed: 0,user_id,nickname,total_kills,total_deaths,total_time_played,total_kills_headshot,total_shots_hit,total_mvps,total_wins,...,ratio_hit_kill,ratio_hit_fired,ratio_win_round,lang,country,sniper_score,assault_score,support_score,captain_score,strategist_score
0,0,56371,BraydenBVR,7206.0,6521.0,39744916.0,2873.0,23565.0,825.0,2977.0,...,0.305792,0.188986,0.497743,en,us,0.777509,0.808472,0.047330,0.691375,0.803528
1,1,66955,LittleBigFg,15507.0,12568.0,34471515.0,3481.0,40941.0,1067.0,7414.0,...,0.378765,0.233820,0.853361,ru,ru,0.764931,0.835938,0.816492,0.599836,0.311619
2,2,4042,VACooo,57221.0,55091.0,28495214.0,21566.0,147147.0,6566.0,24903.0,...,0.388870,0.144602,0.573934,ru,ge,0.121570,0.727647,0.219712,0.030917,0.741702
3,3,87560,EventGraph,198934.0,170105.0,28147897.0,115160.0,366917.0,38962.0,1306899.0,...,0.542177,0.573855,0.527337,sv,se,0.811519,0.266277,0.888977,0.676056,0.502791
5,5,22706,0rdain,5649.0,4501.0,26247390.0,2451.0,17386.0,605.0,2528.0,...,0.324917,0.209599,0.548611,en,ca,0.737199,0.609903,0.362983,0.620232,0.367440


In [25]:
def graph_connectdness(team_meamber_id,free_member_id,nb_pre_selection=50,data=df):
    # Return the list of the 40 people the most connected to the team given individual_performances
    
    df_ratio=pd.DataFrame(data[data.user_id.isin(free_member_id)][['ratio_headshot','ratio_hit_kill','ratio_hit_fired','ratio_win_round']]             
         )
    team_player_=data[data.user_id.isin(team_meamber_id)][['ratio_headshot','ratio_hit_kill','ratio_hit_fired','ratio_win_round']]
    e_d=euclidean_distances(team_player_.as_matrix(),df_ratio.as_matrix())
    weight=len(e_d[0])-np.argsort(np.argsort(e_d))
    k=np.argsort(np.sum(weight,axis=0))
   
    return  free_member_id[k[len(k)-nb_pre_selection:]]



def graph_connectdness_boo(team_meamber_id,free_member_id,nb_pre_selection=400,data=df):
    
    #return the list of the 200 people the most connected in term of lang, country, rank_id and and goal_id 
    cols_to_transform =['lang','country','rank_id','goal_id']
    df = pd.get_dummies(data,columns=cols_to_transform)
    df_ratio=data[data.user_id.isin(free_member_id)].iloc[:,24:]             
    team_player_=data[data.user_id.isin(team_meamber_id)].iloc[:,24:] 
    test=pairwise_distances(team_player_.as_matrix(),df_ratio.as_matrix(), metric='manhattan')

    return free_member_id[np.argsort(np.sum(test,axis=0))[:nb_pre_selection]]

def set_recommandation(team_meamber_id,free_member_id,nb_recommmand=4,pre_selection=2,data=df):
    if pre_selection==2:
        free_member_id=graph_connectdness_boo(team_meamber_id,free_member_id,nb_pre_selection=400)
       
        free_member_id=graph_connectdness(team_meamber_id,free_member_id,nb_pre_selection=40)
    if pre_selection==1:
         free_member_id=graph_connectdness(team_meamber_id,free_member_id,nb_pre_selection=40)
        
    team_player_=data[data.user_id.isin(team_meamber_id)][['sniper_score','assault_score','captain_score','strategist_score']]
    df_ratio=pd.DataFrame(data[data.user_id.isin(free_member_id)][['sniper_score','assault_score','captain_score','strategist_score']])
    feature_to_improve = team_player_.mean().idxmin()
    def_score=[]
    for i,j in enumerate(free_member_id):
        score=team_player_.append(df_ratio.iloc[i]).mean()[feature_to_improve]
        def_score.append(score)
    return free_member_id[np.argsort(def_score)[(len(free_member_id)-nb_recommmand-1):]]


In [ ]:
 print(df_ratio)
    team_player_=data[data.user_id.isin(team_meamber_id)][['ratio_headshot','ratio_hit_kill','ratio_hit_fired','ratio_win_round']]
    print(    team_player_)
    e_d=euclidean_distances(team_player_.as_matrix(),df_ratio.as_matrix())
    print(team_player_.shape)
    print(e_d.shape)
    weight=len(e_d[0])-np.argsort(np.argsort(e_d))

    k=np.argsort(np.sum(weight,axis=0))

## Example

In [27]:
player=np.random.choice(list(set(df.user_id)),3000,replace=False)
free_player = player[:2996]
team_player = player[2996:]
set_recommandation(team_player,free_player,pre_selection=2)

        ratio_headshot  ratio_hit_kill  ratio_hit_fired  ratio_win_round
274           0.579503        0.317492         0.206185         0.553369
566           0.410918        0.378716         0.193832         0.517358
570           0.391749        0.334304         0.213586         0.507778
1007          0.452967        0.339144         0.175071         0.509181
1375          0.576082        0.305364         0.205117         0.502886
2293          0.371048        0.344893         0.191950         0.526221
2461          0.333567        0.295012         0.135429         0.500871
3566          0.648449        0.331438         0.156064         0.518824
3729          0.419080        0.345088         0.168343         0.504393
3766          0.347248        0.346731         0.193497         0.491422
4152          0.506955        0.286764         0.148227         0.501066
4192          0.442344        0.498778         0.182966         0.528552
4356          0.406674        0.342171         0.17

array([92954, 38901, 49865, 64467, 14543])

In [9]:
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.feature_extraction.text import CountVectorizer

corpus = ['UNC played Duke in basketball','Duke lost the basketball game','I ate a sandwich']# 文集  
vectorizer = CountVectorizer()# 
counts=vectorizer.fit_transform(corpus).todense()
for x,y in [[0,1],[0,2],[1,2]]:
    dist=euclidean_distances(counts[x],counts[y])
    print('文档{}与文档{}的距离{}'.format(x,y,dist))

文档0与文档1的距离[[ 2.44948974]]
文档0与文档2的距离[[ 2.64575131]]
文档1与文档2的距离[[ 2.64575131]]
